In [2]:
import os
import csv
import adin.settings as st
from IPython.display import display
from references.forms.address_forms import AddressCreateModelForm
from os.path import join
from adin.settings import BASE_DIR

from scripts.utils import m2m_info, m2m_load, m2m_backup

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
def data_load(load_info, load_list=None):
    counter = 0
    timers = { counter:datetime.now() }
    if not load_list:
        load_list = load_lists['all']
    for element in load_list:
        model_load(load_info[element])
        counter = counter + 1
        timers[counter] = datetime.now()
        print(f"{element} : {timers[counter] - timers[counter - 1]}")
    
def model_load(load_dict, csv_file=None):
    df_from_csv = pd.read_csv(csv_file if csv_file else join(BASE_DIR, f"_files/{load_dict['csv_name']}"), keep_default_na=False)\
        .drop(load_dict['to_drop'], axis=1)\
        .rename(columns=load_dict['to_rename'])
    data_dict = df_from_csv\
        .assign(**{column: df_from_csv[column].apply(lambda x: None if x == '' else x) for column in df_from_csv.columns})\
        .assign(**{column: df_from_csv[column].apply(lambda x: None if x == '' else model.objects.get(pk=x)) for column, model in load_dict['fk_dict'].items()})\
        .to_dict('index')
    if load_dict['bulk']:
        load_dict['model'].objects.bulk_create([load_dict['model'](**data) for data in data_dict.values()])
    else:
        for data in data_dict.values():
            load_dict['model'](**data).save()
    if load_dict['pending_relations']:
        for relation in load_dict['pending_relations']:
            m2m_load(m2m_info[relation])

def m2m_load(load_dict):
    df_from_csv = pd.read_csv(join(BASE_DIR, f"_files/{load_dict['csv_name']}"), keep_default_na=False)
    df_from_csv.assign(**{load_dict['field_obj_column']:df_from_csv[load_dict['field_obj_column']].apply(lambda x: load_dict['field_obj_model'].objects.get(pk=x))})\
        .apply(lambda x: eval(f"x[load_dict['field_obj_column']].{load_dict['m2m_field']}.add(load_dict['non_field_obj_model'].objects.get(pk=x[load_dict['non_field_obj_column']]))"), axis=1)

In [4]:
gr=Group.objects.all()[0]
gr

<Group: admin>

In [5]:
dir(gr)

['DoesNotExist',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_default_pk',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check_unique_together',
 '_do_insert',
 '_do_update',
 '_get_

In [5]:
pd.DataFrame(Group.objects.all().values('pk', 'permissions'))

,pk,permissions
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
162,1,163
163,1,164
164,1,165
165,1,166


In [32]:
pd.DataFrame(Lease_Realty.objects.all().values('pk', 'transaction_types'))

,pk,transaction_types
0,A6N-13N-62-S101^2021-04-01,Arriendo Inmueble Comercial
1,A6N-13N-62-S101^2021-04-01,Arriendo Inmueble Residencial
2,A6N-13N-62-S102^2021-10-01,Arriendo Inmueble Residencial
3,A6N-13N-62-S103^2020-06-01,Arriendo Inmueble Residencial
4,A6N-13N-62-S104^2021-08-01,Arriendo Inmueble Residencial
...,...,...
69,K2C-30-15-B2^2019-07-01,Arriendo Inmueble Comercial
70,K2C-30-15-B3^2013-06-01,Arriendo Inmueble Comercial
71,K2C-30-27^2020-10-01,Arriendo Inmueble Comercial
72,K2C-30-29^2019-11-01,Arriendo Inmueble Comercial


In [3]:
load_dict=m2m_info['user_permissions']
df=pd.DataFrame(load_dict['field_obj_model'].objects.all().values('pk', load_dict['m2m_field']))
df=df.rename(columns={'pk':load_dict['field_obj_column'], load_dict['m2m_field']:load_dict['non_field_obj_column']})
df

,user,user_permission
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
162,1,163
163,1,164
164,1,165
165,1,166


In [3]:
m2m_backup(m2m_info['user_permissions'])

In [6]:
user=User.objects.all()[0]

In [8]:
dir(user)

['DoesNotExist',
 'EMAIL_FIELD',
 'Meta',
 'MultipleObjectsReturned',
 'REQUIRED_FIELDS',
 'USERNAME_FIELD',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_default_pk',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',


In [11]:
df_from_csv = pd.read_csv(join(BASE_DIR, f"_files/load/user.csv"), na_values='NaT', keep_default_na=False)\
        .drop(['Unnamed: 0'], axis=1)
df_from_csv

,id,password,last_login,is_superuser,username,first_name,last_name,email,is_staff,is_active,date_joined
0,2,pbkdf2_sha256$320000$AlPghfRXboxGEwzOpqX2W0$qh...,NaT,False,jaque,,,,False,True,2022-07-12 17:44:22.510737+00:00
1,3,pbkdf2_sha256$320000$ddhdIcwWcKvi0sLRdbOJzX$Gj...,NaT,False,yaneth,,,,False,True,2022-07-12 17:44:38.512188+00:00
